# Chapter 4: RF Point Positioning

## Principles of Indoor Positioning and Indoor Navigation

---

### 📚 Learning Objectives

By the end of this notebook, you will be able to:

1. **Understand** different RF positioning techniques (TOA, TDOA, AOA, RSS)
2. **Implement** Time-of-Arrival (TOA) positioning using iterative least squares
3. **Apply** Time Difference of Arrival (TDOA) for clock-bias-free positioning
4. **Use** Angle of Arrival (AOA) and triangulation methods
5. **Analyze** the impact of geometry on positioning accuracy (GDOP/HDOP)
6. **Compare** performance of different RF methods under various noise conditions

### 📖 Book Reference

This notebook covers **Chapter 4: Point Positioning by Radio Signals** with:
- **Eq. (4.1)-(4.3)**: TOA range measurements
- **Eq. (4.11)-(4.13)**: RSS path-loss model
- **Eq. (4.14)-(4.23)**: Nonlinear TOA I-WLS positioning
- **Eq. (4.27)-(4.42)**: TDOA positioning
- **Eq. (4.63)-(4.67)**: AOA positioning
- **Section 4.5**: DOP and geometry analysis

---


## 🚀 Setup (Google Colab)

**Set the `GITHUB_REPO` variable below to your repository URL, then run the setup cell.**

Example: `GITHUB_REPO = "https://github.com/YOUR_USERNAME/IPIN_Book_Examples.git"`


In [ ]:
# ========================================
# IPIN Book Examples - Chapter 4: RF Positioning
# ========================================

import os
import sys

# ============ CONFIGURATION ============
GITHUB_REPO = None  # Set your repo URL, e.g., "https://github.com/username/IPIN_Book_Examples.git"
# =======================================

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    if os.path.exists('/content/IPIN_Book_Examples/core'):
        os.chdir('/content/IPIN_Book_Examples')
        print("✅ Repository already available.")
    elif GITHUB_REPO:
        print(f"📥 Cloning from {GITHUB_REPO}...")
        get_ipython().system(f'git clone {GITHUB_REPO}')
        os.chdir('/content/IPIN_Book_Examples')
        get_ipython().system('pip install -e . -q')
        print("✅ Setup from GitHub complete!")
    else:
        print("❌ ERROR: GITHUB_REPO not set!")
        print("Please set GITHUB_REPO = 'https://github.com/YOUR_USERNAME/IPIN_Book_Examples.git'")
        raise ValueError("GITHUB_REPO not configured.")
else:
    if os.path.basename(os.getcwd()) == 'notebooks':
        os.chdir('..')
    print(f"📂 Working directory: {os.getcwd()}")

# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle

get_ipython().run_line_magic('matplotlib', 'inline')
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100

# Import RF positioning modules
from core.rf import (
    TOAPositioner,
    TDOAPositioner,
    AOAPositioner,
    toa_range,
    rss_pathloss,
    rss_to_distance,
    aoa_azimuth,
)

print("\n" + "="*60)
print("✅ Setup complete! RF positioning modules loaded.")
print("="*60)


---

# Part 1: RF Positioning Overview

## 1.1 Positioning Methods Summary

| Method | Measurement | Equation | Pros | Cons |
|--------|-------------|----------|------|------|
| **TOA** | Range (time × speed of light) | Eq. 4.1-4.3 | High accuracy | Requires synchronized clocks |
| **TDOA** | Range differences | Eq. 4.27-4.33 | No agent clock sync | Anchor sync required |
| **AOA** | Azimuth/elevation angles | Eq. 4.63-4.67 | Only 2 anchors needed (2D) | Antenna array required |
| **RSS** | Signal strength → distance | Eq. 4.11-4.13 | Simple hardware | Low accuracy (path-loss uncertainty) |

## 1.2 Common Setup: Anchor Geometry

We'll use a **square anchor configuration** (10m × 10m) as a baseline for all examples.


In [ ]:
# Common setup: Anchor geometry and test positions
print("="*70)
print("Setting Up Test Scenario")
print("="*70)

# Square anchor layout (4 corners of 10m x 10m room)
anchors = np.array([
    [0.0, 0.0],    # A1: Southwest corner
    [10.0, 0.0],   # A2: Southeast corner
    [10.0, 10.0],  # A3: Northeast corner
    [0.0, 10.0],   # A4: Northwest corner
], dtype=float)

# True agent position (center of room)
true_pos = np.array([5.0, 5.0])

print(f"\n🏢 Anchor positions (4 corners):")
for i, anchor in enumerate(anchors):
    print(f"  A{i+1}: ({anchor[0]:.1f}, {anchor[1]:.1f}) m")
print(f"\n🎯 True agent position: ({true_pos[0]:.1f}, {true_pos[1]:.1f}) m")

# Visualize setup
fig, ax = plt.subplots(figsize=(8, 8))

# Plot anchors
ax.scatter(anchors[:, 0], anchors[:, 1], s=200, c='red', marker='^', 
           label='Anchors', zorder=5)
for i, anchor in enumerate(anchors):
    ax.text(anchor[0], anchor[1] + 0.5, f'A{i+1}', ha='center', 
            fontsize=12, fontweight='bold')

# Plot true position
ax.scatter(true_pos[0], true_pos[1], s=150, c='green', marker='o', 
           label='True Position', zorder=5)

ax.set_xlim(-1, 11)
ax.set_ylim(-1, 11)
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
ax.set_xlabel('East (m)', fontsize=12)
ax.set_ylabel('North (m)', fontsize=12)
ax.set_title('RF Positioning Scenario: 10m × 10m Room', fontsize=14, fontweight='bold')
ax.legend(loc='upper right')
plt.tight_layout()
plt.show()


---

# Part 2: TOA (Time of Arrival) Positioning

## 2.1 Theory

**TOA** measures the time for a signal to travel from anchor to agent:

$$r_i = c \cdot t_i = \| \mathbf{p} - \mathbf{a}_i \|$$

where:
- $r_i$ = range to anchor $i$
- $c$ = speed of light (≈ 3×10⁸ m/s)
- $\mathbf{p}$ = agent position (unknown)
- $\mathbf{a}_i$ = anchor $i$ position (known)

**Solution**: Iterative Weighted Least Squares (I-WLS) minimizes:

$$\min_{\mathbf{p}} \sum_i w_i \left( r_i - \| \mathbf{p} - \mathbf{a}_i \| \right)^2$$


In [ ]:
# Example 1: TOA Positioning with Perfect Measurements
print("="*70)
print("Example 1: TOA Positioning (Perfect Measurements)")
print("="*70)

# Compute true ranges from anchors to agent
true_ranges = np.array([toa_range(anchor, true_pos) for anchor in anchors])

print(f"\n📏 True ranges to each anchor:")
for i, (anchor, r) in enumerate(zip(anchors, true_ranges)):
    print(f"  A{i+1}: {r:.4f} m")

# Solve using I-WLS
positioner = TOAPositioner(anchors, method='iwls')
initial_guess = np.array([6.0, 6.0])  # Start from offset position
estimated_pos, info = positioner.solve(true_ranges, initial_guess=initial_guess)

# Results
error = np.linalg.norm(estimated_pos - true_pos)
print(f"\n🎯 Results:")
print(f"  True position:      ({true_pos[0]:.4f}, {true_pos[1]:.4f}) m")
print(f"  Estimated position: ({estimated_pos[0]:.4f}, {estimated_pos[1]:.4f}) m")
print(f"  Position error:     {error:.6f} m")
print(f"  Converged:          {info['converged']}")
print(f"  Iterations:         {info['iterations']}")


In [ ]:
# Example 2: TOA Positioning with Measurement Noise
print("="*70)
print("Example 2: TOA Positioning (With 10cm Noise)")
print("="*70)

np.random.seed(42)

# Add Gaussian noise to ranges
noise_std = 0.10  # 10 cm standard deviation
noisy_ranges = true_ranges + np.random.randn(len(anchors)) * noise_std

print(f"\n📏 Range measurements (σ = {noise_std*100:.0f} cm):")
for i, (true_r, noisy_r) in enumerate(zip(true_ranges, noisy_ranges)):
    error_r = noisy_r - true_r
    print(f"  A{i+1}: True={true_r:.4f}m, Measured={noisy_r:.4f}m, Error={error_r*100:+.1f}cm")

# Solve using I-WLS
estimated_pos_noisy, info_noisy = positioner.solve(noisy_ranges, initial_guess=initial_guess)

# Results
error_noisy = np.linalg.norm(estimated_pos_noisy - true_pos)
print(f"\n🎯 Results:")
print(f"  True position:      ({true_pos[0]:.4f}, {true_pos[1]:.4f}) m")
print(f"  Estimated position: ({estimated_pos_noisy[0]:.4f}, {estimated_pos_noisy[1]:.4f}) m")
print(f"  Position error:     {error_noisy:.4f} m")
print(f"  Error/Noise ratio:  {error_noisy/noise_std:.2f}×")


In [ ]:
# Visualize TOA positioning
fig, ax = plt.subplots(figsize=(10, 10))

# Plot anchors
ax.scatter(anchors[:, 0], anchors[:, 1], s=200, c='red', marker='^', 
           label='Anchors', zorder=5)
for i, anchor in enumerate(anchors):
    ax.text(anchor[0], anchor[1] + 0.4, f'A{i+1}', ha='center', 
            fontsize=11, fontweight='bold')

# Plot range circles (true ranges)
for anchor, r in zip(anchors, true_ranges):
    circle = Circle(anchor, r, fill=False, color='red', alpha=0.3, 
                    linestyle='--', linewidth=1.5)
    ax.add_patch(circle)

# Plot true position
ax.scatter(true_pos[0], true_pos[1], s=150, c='green', marker='o', 
           label='True Position', zorder=5)

# Plot estimated position (noisy)
ax.scatter(estimated_pos_noisy[0], estimated_pos_noisy[1], s=150, c='blue', 
           marker='x', linewidths=3, label=f'Estimated (error={error_noisy:.3f}m)', zorder=5)

# Plot convergence path if available
if 'history' in info_noisy and info_noisy['history'] is not None:
    history = info_noisy['history']
    ax.plot(history[:, 0], history[:, 1], 'b--', alpha=0.5, 
            label='Convergence Path', zorder=3)

ax.set_xlim(-2, 12)
ax.set_ylim(-2, 12)
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
ax.set_xlabel('East (m)', fontsize=12)
ax.set_ylabel('North (m)', fontsize=12)
ax.set_title('TOA Positioning with Range Circles', fontsize=14, fontweight='bold')
ax.legend(loc='upper right')
plt.tight_layout()
plt.show()

print("\n💡 Key Insight: The true position is where all range circles intersect!")


---

# Part 3: TDOA (Time Difference of Arrival) Positioning

## 3.1 Theory

**TDOA** eliminates the need for agent clock synchronization by measuring **range differences**:

$$\Delta r_{i,0} = r_i - r_0 = \| \mathbf{p} - \mathbf{a}_i \| - \| \mathbf{p} - \mathbf{a}_0 \|$$

where:
- $\Delta r_{i,0}$ = range difference between anchor $i$ and reference anchor 0
- This defines a **hyperbola** with foci at $\mathbf{a}_0$ and $\mathbf{a}_i$

**Advantage**: Agent doesn't need synchronized clock (clock bias cancels out)


In [ ]:
# Example 3: TDOA Positioning
print("="*70)
print("Example 3: TDOA Positioning")
print("="*70)

# Compute TDOA measurements (range differences relative to anchor 0)
reference_idx = 0
dist_ref = np.linalg.norm(true_pos - anchors[reference_idx])
tdoa_measurements = np.array([
    np.linalg.norm(true_pos - anchors[i]) - dist_ref 
    for i in range(1, len(anchors))
])

print(f"\n📏 TDOA measurements (relative to A{reference_idx+1}):")
for i, tdoa in enumerate(tdoa_measurements, start=1):
    print(f"  Δr_{i+1},{reference_idx+1} = {tdoa:+.4f} m")

# Add noise
np.random.seed(42)
tdoa_noise_std = 0.10
tdoa_noisy = tdoa_measurements + np.random.randn(len(tdoa_measurements)) * tdoa_noise_std

print(f"\n📏 Noisy TDOA measurements (σ = {tdoa_noise_std*100:.0f} cm):")
for i, (true_t, noisy_t) in enumerate(zip(tdoa_measurements, tdoa_noisy), start=1):
    print(f"  Δr_{i+1},{reference_idx+1} = {noisy_t:+.4f} m (error: {(noisy_t-true_t)*100:+.1f} cm)")

# Solve using TDOA positioner
tdoa_positioner = TDOAPositioner(anchors, reference_idx=reference_idx)
estimated_pos_tdoa, info_tdoa = tdoa_positioner.solve(
    tdoa_noisy, 
    initial_guess=np.array([6.0, 6.0])
)

# Results
error_tdoa = np.linalg.norm(estimated_pos_tdoa - true_pos)
print(f"\n🎯 Results:")
print(f"  True position:      ({true_pos[0]:.4f}, {true_pos[1]:.4f}) m")
print(f"  Estimated position: ({estimated_pos_tdoa[0]:.4f}, {estimated_pos_tdoa[1]:.4f}) m")
print(f"  Position error:     {error_tdoa:.4f} m")


---

# Part 4: AOA (Angle of Arrival) Positioning

## 4.1 Theory

**AOA** measures the direction from anchor to agent using antenna arrays:

$$\theta_i = \arctan\left(\frac{p_y - a_{i,y}}{p_x - a_{i,x}}\right)$$

where:
- $\theta_i$ = azimuth angle from anchor $i$ to agent
- Each measurement defines a **bearing line**

**Advantage**: Only 2 anchors needed in 2D (triangulation)

**Disadvantage**: Requires antenna array hardware, accuracy degrades with distance


In [ ]:
# Example 4: AOA Positioning
print("="*70)
print("Example 4: AOA Positioning")
print("="*70)

# Compute true AOA measurements (azimuth angles)
true_aoa = np.array([aoa_azimuth(anchor, true_pos) for anchor in anchors])

print(f"\n📐 True AOA measurements (azimuth angles):")
for i, (anchor, theta) in enumerate(zip(anchors, true_aoa)):
    print(f"  A{i+1}: θ = {np.rad2deg(theta):+.2f}°")

# Add noise
np.random.seed(42)
aoa_noise_std_deg = 3.0  # 3 degrees
aoa_noise_std_rad = np.deg2rad(aoa_noise_std_deg)
aoa_noisy = true_aoa + np.random.randn(len(anchors)) * aoa_noise_std_rad

print(f"\n📐 Noisy AOA measurements (σ = {aoa_noise_std_deg:.0f}°):")
for i, (true_a, noisy_a) in enumerate(zip(true_aoa, aoa_noisy)):
    error_deg = np.rad2deg(noisy_a - true_a)
    print(f"  A{i+1}: θ = {np.rad2deg(noisy_a):+.2f}° (error: {error_deg:+.2f}°)")

# Solve using AOA positioner
aoa_positioner = AOAPositioner(anchors)
estimated_pos_aoa, info_aoa = aoa_positioner.solve(
    aoa_noisy, 
    initial_guess=np.array([6.0, 6.0])
)

# Results
error_aoa = np.linalg.norm(estimated_pos_aoa - true_pos)
print(f"\n🎯 Results:")
print(f"  True position:      ({true_pos[0]:.4f}, {true_pos[1]:.4f}) m")
print(f"  Estimated position: ({estimated_pos_aoa[0]:.4f}, {estimated_pos_aoa[1]:.4f}) m")
print(f"  Position error:     {error_aoa:.4f} m")


---

# Part 5: RSS (Received Signal Strength) Positioning

## 5.1 Theory

**RSS** uses the log-distance path-loss model to estimate distance:

$$P_{rx} = P_{tx} - 10 \cdot n \cdot \log_{10}\left(\frac{d}{d_0}\right)$$

where:
- $P_{rx}$ = received power (dBm)
- $P_{tx}$ = transmitted power (dBm)
- $n$ = path-loss exponent (≈2-4 indoors)
- $d$ = distance, $d_0$ = reference distance (1m)

**Advantage**: Simple hardware (any WiFi/BLE device)

**Disadvantage**: High uncertainty due to multipath, shadowing, and unknown path-loss exponent


In [ ]:
# Example 5: RSS-Based Ranging and Positioning
print("="*70)
print("Example 5: RSS-Based Positioning")
print("="*70)

# RSS parameters
tx_power_dbm = 0.0    # Transmit power (dBm)
path_loss_exp = 2.5   # Path-loss exponent (indoor)

print(f"\n📡 RSS Parameters:")
print(f"  Tx power: {tx_power_dbm} dBm")
print(f"  Path-loss exponent: {path_loss_exp}")

# Compute RSS at each anchor
print(f"\n📏 RSS measurements at each anchor:")
rss_measurements = []
for i, anchor in enumerate(anchors):
    dist = np.linalg.norm(true_pos - anchor)
    rss = rss_pathloss(tx_power_dbm, dist, path_loss_exp)
    rss_measurements.append(rss)
    print(f"  A{i+1}: Distance = {dist:.2f}m → RSS = {rss:.2f} dBm")

# Convert RSS back to ranges
ranges_from_rss = np.array([
    rss_to_distance(rss, tx_power_dbm, path_loss_exp) 
    for rss in rss_measurements
])

print(f"\n📏 Ranges estimated from RSS:")
for i, (true_r, est_r) in enumerate(zip(true_ranges, ranges_from_rss)):
    error_pct = 100 * (est_r - true_r) / true_r
    print(f"  A{i+1}: True={true_r:.2f}m, Estimated={est_r:.2f}m (error: {error_pct:+.1f}%)")

# Position using RSS-derived ranges
estimated_pos_rss, info_rss = positioner.solve(ranges_from_rss, initial_guess=np.array([6.0, 6.0]))
error_rss = np.linalg.norm(estimated_pos_rss - true_pos)

print(f"\n🎯 Results (perfect RSS, no fading):")
print(f"  Position error: {error_rss:.4f} m")
print(f"\n⚠️ Note: Real RSS has ~3-6 dB shadowing noise, causing meter-level errors!")


---

# Part 6: Method Comparison

## 6.1 Monte Carlo Simulation

Let's compare all methods under identical noise conditions across multiple trials.


In [ ]:
# Monte Carlo comparison of RF positioning methods
print("="*70)
print("Monte Carlo Comparison: 100 Trials")
print("="*70)

np.random.seed(42)
n_trials = 100

# Noise levels
range_noise_std = 0.10  # 10 cm for TOA/TDOA
aoa_noise_std = np.deg2rad(3.0)  # 3 degrees for AOA

# Results storage
results = {
    'TOA': [],
    'TDOA': [],
    'AOA': [],
}

for trial in range(n_trials):
    # Generate noisy measurements
    noisy_ranges = true_ranges + np.random.randn(len(anchors)) * range_noise_std
    noisy_tdoa = tdoa_measurements + np.random.randn(len(tdoa_measurements)) * range_noise_std
    noisy_aoa = true_aoa + np.random.randn(len(anchors)) * aoa_noise_std
    
    # TOA
    try:
        est, info = positioner.solve(noisy_ranges, initial_guess=np.mean(anchors, axis=0))
        if info['converged']:
            results['TOA'].append(np.linalg.norm(est - true_pos))
    except:
        pass
    
    # TDOA
    try:
        est, info = tdoa_positioner.solve(noisy_tdoa, initial_guess=np.mean(anchors, axis=0))
        if info['converged']:
            results['TDOA'].append(np.linalg.norm(est - true_pos))
    except:
        pass
    
    # AOA
    try:
        est, info = aoa_positioner.solve(noisy_aoa, initial_guess=np.mean(anchors, axis=0))
        if info['converged']:
            results['AOA'].append(np.linalg.norm(est - true_pos))
    except:
        pass

# Print summary
print(f"\n📊 Results Summary (range noise={range_noise_std*100:.0f}cm, AOA noise=3°):\n")
print(f"{'Method':<8} {'RMSE (m)':<10} {'Mean (m)':<10} {'Max (m)':<10} {'Success':<10}")
print("-" * 48)

for method in ['TOA', 'TDOA', 'AOA']:
    errors = np.array(results[method])
    if len(errors) > 0:
        rmse = np.sqrt(np.mean(errors**2))
        mean_err = np.mean(errors)
        max_err = np.max(errors)
        success = len(errors) / n_trials * 100
        print(f"{method:<8} {rmse:<10.4f} {mean_err:<10.4f} {max_err:<10.4f} {success:<10.0f}%")
    else:
        print(f"{method:<8} {'N/A':<10} {'N/A':<10} {'N/A':<10} 0%")


In [ ]:
# Visualize comparison results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Box plot
ax1 = axes[0]
data_to_plot = [results['TOA'], results['TDOA'], results['AOA']]
labels = ['TOA', 'TDOA', 'AOA']
colors = ['#3498db', '#e74c3c', '#2ecc71']

bp = ax1.boxplot(data_to_plot, labels=labels, patch_artist=True)
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax1.set_ylabel('Position Error (m)', fontsize=12)
ax1.set_title('Error Distribution by Method', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='y')

# CDF plot
ax2 = axes[1]
for method, color in zip(['TOA', 'TDOA', 'AOA'], colors):
    errors = np.sort(results[method])
    cdf = np.arange(1, len(errors) + 1) / len(errors)
    ax2.plot(errors, cdf, label=method, color=color, linewidth=2)

ax2.axhline(y=0.95, color='gray', linestyle='--', alpha=0.7, label='95th percentile')
ax2.set_xlabel('Position Error (m)', fontsize=12)
ax2.set_ylabel('CDF', fontsize=12)
ax2.set_title('Cumulative Distribution Function', fontsize=14, fontweight='bold')
ax2.legend(loc='lower right')
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, None)

plt.tight_layout()
plt.show()

print("\n💡 Key Observations:")
print("  - TOA and TDOA achieve similar accuracy with good geometry")
print("  - AOA accuracy depends on distance to anchors")
print("  - All methods benefit from symmetric anchor placement")


---

# Summary

## Key Takeaways

### 1. Method Selection Guide

| Scenario | Recommended Method | Reason |
|----------|-------------------|--------|
| High accuracy, sync available | **TOA** | Best with synchronized clocks |
| No agent clock sync | **TDOA** | Clock bias cancels out |
| Limited anchors (2-3) | **AOA** | Triangulation with fewer beacons |
| Simple hardware only | **RSS** | Works with any WiFi/BLE |

### 2. Accuracy Hierarchy (typical indoor)

| Method | Typical Accuracy | Requirements |
|--------|-----------------|--------------|
| UWB TOA | 10-30 cm | Synchronized clocks |
| UWB TDOA | 20-50 cm | Anchor synchronization |
| BLE AOA | 50-100 cm | Antenna arrays |
| WiFi RSS | 1-5 m | Signal strength only |

### 3. Geometry Matters (GDOP)

- **Good**: Anchors surrounding target (square, circular)
- **Bad**: Anchors in a line → poor GDOP → large errors
- HDOP multiplies measurement noise: $\sigma_{pos} \approx \text{HDOP} \times \sigma_{range}$

---

## Exercises

1. **Modify anchor geometry**: Try linear placement (all on one wall) - what happens to accuracy?
2. **Increase noise**: How does 20cm range noise affect TOA vs TDOA?
3. **Fewer anchors**: Can you achieve sub-meter accuracy with only 3 anchors?

---

**Next Steps:** Chapter 5 (Fingerprinting) for signal-strength-based positioning without path-loss models!
